# Database Search

*By Hannes Hedberg*

---

## 1. Pyodbc

Reading alot about Pyodbc I managed to solve the assignment in pyodbc, before i realised it *had* to be solved using SQLAlchemy, so I decided to include it below. 

In [1]:
import pyodbc

def search_books_by_title(title):
    # Establish a connection to the database
    server = 'Hannes-PC'
    database = 'HannesHedberg'
    connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
    conn = pyodbc.connect(connection_string)

    # Create a cursor object
    cursor = conn.cursor()

    # Prepare the SQL query
    query = """
    SELECT b.Title, s.Name AS StoreName, i.Num AS Number
    FROM Book b
    JOIN Inventory i ON i.ISBN13 = b.ISBN13
    JOIN Store s ON s.StoreID = i.StoreID
    WHERE b.Title LIKE ?
    """

    # Execute the query with the provided title parameter
    cursor.execute(query, ('%' + title + '%',))

    # Fetch all the rows as a list of tuples
    rows = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    # Return the search results
    return rows

# Example usage: Search for books by title and get available copies in each store
search_title = input("Enter a book title to search: ")
books = search_books_by_title(search_title)

# Display the search results
for book in books:
    print("Title:", book.Title)
    print("Store Name:", book.StoreName)
    print("Number of Copies:", book.Number)
    print()

Title: Harry Potter and the Deathly Hallows
Store Name: The Page Sage
Number of Copies: 9

Title: Harry Potter and the Prisoner of Azkaban
Store Name: The Page Sage
Number of Copies: 10

Title: Harry Potter and the Philosopher's Stone
Store Name: Once Upon A Time
Number of Copies: 9

Title: Harry Potter and the Chamber of Secrets
Store Name: Once Upon A Time
Number of Copies: 9

Title: Harry Potter and the Goblet of Fire
Store Name: Turn The Page
Number of Copies: 10

Title: Harry Potter and the Half-Blood Prince
Store Name: Turn The Page
Number of Copies: 15

Title: Harry Potter and the Order of Phoenix
Store Name: The Lore Store
Number of Copies: 11



---

## 2. SQL - Alchemy

Solving it using SQLAlchemy instead, and it seems to be working most of the time. But sometimes it gives me a driver error, not sure whats causing this. If im using outdated code, and perhaps some of it is no longer included in the latest versions? Or if my own software is outdated. If there is an error when you run it please help me understand why.

In [2]:
from sqlalchemy import create_engine, MetaData, Table, select

def search_books_by_title(title):
    # Establish a connection to the database
    server = 'Hannes-PC'
    database = 'HannesHedberg'
    connection_string = f"mssql+pyodbc://{server}/{database}?driver=SQL+Server&Trusted_Connection=yes&fast_executemany=True"
    engine = create_engine(connection_string)

    # Define metadata and reflect the tables
    metadata = MetaData()
    metadata.reflect(bind=engine)

    # Get the table objects
    Book = Table('Book', metadata, autoload=True)
    Inventory = Table('Inventory', metadata, autoload=True)
    Store = Table('Store', metadata, autoload=True)

    # Prepare the SQL query
    query = select([Book.c.Title, Store.c.Name.label('StoreName'), Inventory.c.Num.label('Number')]) \
        .select_from(Book.join(Inventory, Book.c.ISBN13 == Inventory.c.ISBN13)
                     .join(Store, Store.c.StoreID == Inventory.c.StoreID)) \
        .where(Book.c.Title.like(f'%{title}%'))

    # Execute the query
    with engine.connect() as conn:
        result = conn.execute(query)
        rows = result.fetchall()

    # Return the search results
    return rows

# Example usage: Search for books by title and get available copies in each store
search_title = input("Enter a book title to search: ")
books = search_books_by_title(search_title)

# Display the search results
for book in books:
    print("Title:", book.Title)
    print("Store Name:", book.StoreName)
    print("Number of Copies:", book.Number)
    print()

DBAPIError: (pyodbc.Error) ('HY104', '[HY104] [Microsoft][ODBC SQL Server Driver]Ogiltigt precisionsvärde (0) (SQLBindParameter)')
[SQL: SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]]
[parameters: ('dbo', 'BASE TABLE')]
(Background on this error at: https://sqlalche.me/e/20/dbapi)